#Implementing the Variational Quantum Eigensolver using tket and qiskit


In [ ]:
from openfermion import QubitOperator

#defining the hamiltonian for which we are trying to find the ground state.
hamiltonian = (-0.8153001706270075 * QubitOperator('') +
    0.16988452027940318 * QubitOperator('Z0') +
    -0.21886306781219608 * QubitOperator('Z1') +
    0.16988452027940323 * QubitOperator('Z2') +
    -0.2188630678121961 * QubitOperator('Z3') +
    0.12005143072546047 * QubitOperator('Z0 Z1') +
    0.16821198673715723 * QubitOperator('Z0 Z2') +
    0.16549431486978672 * QubitOperator('Z0 Z3') +
    0.16549431486978672 * QubitOperator('Z1 Z2') +
    0.1739537877649417 * QubitOperator('Z1 Z3') +
    0.12005143072546047 * QubitOperator('Z2 Z3') +
    0.04544288414432624 * QubitOperator('X0 X1 X2 X3') +
    0.04544288414432624 * QubitOperator('X0 X1 Y2 Y3') +
    0.04544288414432624 * QubitOperator('Y0 Y1 X2 X3') +
    0.04544288414432624 * QubitOperator('Y0 Y1 Y2 Y3'))
nuclear_repulsion_energy = 0.70556961456

In [ ]:
#definining the ansatz (hardwar efficient)
from pytket import Circuit

def hea(params):
    ansatz = Circuit(4)
    for i in range(4) :
        ansatz.Ry(params[i], i)
    for i in range(3) :
        ansatz.CX(i, i+1)
    for i in range(4) :
        ansatz.Ry(params[4+i], i)
    return ansatz

In [ ]:
from pytket.backends.ibm import AerBackend
from pytket.utils import expectation_from_counts

backend = AerBackend()

# creating the optimisation function that uses classical computing to optimise the objective
def objective(params):
    energy = 0
    for term, coeff in hamiltonian.terms.items():
        if not term:
            energy += coeff
            continue
        circ = hea(params)
        circ.add_c_register("c", len(term))
        for i, (q, pauli) in enumerate(term):
            if pauli == 'X':
                circ.H(q)
            elif pauli == 'Y':
                circ.V(q)
            circ.Measure(q, i)
        backend.compile_circuit(circ)
        counts = backend.get_counts(circ, n_shots=4000)
        energy += coeff * expectation_from_counts(counts)
    return energy + nuclear_repulsion_energy

In [ ]:
#inputing the parameter values to retrieve the lowest energy
arg_values = [-7.31158201e-02, -1.64514836e-04, 1.12585591e-03, -2.58367544e-03,
  1.00006068e+00, -1.19551357e-03, 9.99963988e-01, 2.53283285e-03]

energy = objective(arg_values)
print(energy)

-1.1333824130270784


#Repeating the process with tket optimizations

In [ ]:
from pytket.pauli import Pauli, QubitPauliString
from pytket.circuit import Qubit

q = [Qubit(i) for i in range(4)]
xyii = QubitPauliString([q[0], q[1]], [Pauli.X, Pauli.Y])
yxii = QubitPauliString([q[0], q[1]], [Pauli.Y, Pauli.X])
iixy = QubitPauliString([q[2], q[3]], [Pauli.X, Pauli.Y])
iiyx = QubitPauliString([q[2], q[3]], [Pauli.Y, Pauli.X])
xxxy = QubitPauliString(q, [Pauli.X, Pauli.X, Pauli.X, Pauli.Y])
xxyx = QubitPauliString(q, [Pauli.X, Pauli.X, Pauli.Y, Pauli.X])
xyxx = QubitPauliString(q, [Pauli.X, Pauli.Y, Pauli.X, Pauli.X])
yxxx = QubitPauliString(q, [Pauli.Y, Pauli.X, Pauli.X, Pauli.X])
yyyx = QubitPauliString(q, [Pauli.Y, Pauli.Y, Pauli.Y, Pauli.X])
yyxy = QubitPauliString(q, [Pauli.Y, Pauli.Y, Pauli.X, Pauli.Y])
yxyy = QubitPauliString(q, [Pauli.Y, Pauli.X, Pauli.Y, Pauli.Y])
xyyy = QubitPauliString(q, [Pauli.X, Pauli.Y, Pauli.Y, Pauli.Y])

singles_a = {xyii : 1., yxii : -1.}
singles_b = {iixy : 1., iiyx : -1.}
doubles = {
    xxxy : 0.25,
    xxyx : -0.25,
    xyxx : 0.25,
    yxxx : -0.25,
    yyyx : -0.25,
    yxyy : -0.25,
    xyyy : 0.25,
}

In [ ]:
def add_operator_term(circuit : Circuit, term : QubitPauliString, angle : float):
    qubits = []
    for q, p in term.to_dict().items():
        if p != Pauli.I:
            qubits.append(q)
            if p == Pauli.X:
                circuit.H(q)
            elif p == Pauli.Y:
                circuit.V(q)
    for i in range(len(qubits)-1):
        circuit.CX(i, i+1)
    circuit.Rz(angle, len(qubits)-1)
    for i in reversed(range(len(qubits)-1)):
        circuit.CX(i, i+1)
    for q, p in term.to_dict().items():
        if p == Pauli.X:
            circuit.H(q)
        elif p == Pauli.Y:
            circuit.Vdg(q)

# Unitary Coupled Cluster Singles & Doubles ansatz
def ucc(params):
    ansatz = Circuit(4)
    # Set initial reference state
    ansatz.X(1).X(3)
    # Evolve by excitations
    for term, coeff in singles_a.items():
        add_operator_term(ansatz, term, coeff * params[0])
    for term, coeff in singles_b.items():
        add_operator_term(ansatz, term, coeff * params[1])
    for term, coeff in doubles.items():
        add_operator_term(ansatz, term, coeff * params[2])
    return ansatz

In [ ]:
from pytket.circuit import PauliExpBox
from pytket.passes import DecomposeBoxes

def add_excitation(circ, term_dict, param):
    for term, coeff in term_dict.items():
        qubits, paulis = zip(*term.to_dict().items())
        pbox = PauliExpBox(paulis, coeff * param)
        circ.add_pauliexpbox(pbox, qubits)

# UCC ansatz with syntactic shortcuts
def ucc(params):
    ansatz = Circuit(4)
    ansatz.X(1).X(3)
    add_excitation(ansatz, singles_a, params[0])
    add_excitation(ansatz, singles_b, params[1])
    add_excitation(ansatz, doubles, params[2])
    DecomposeBoxes().apply(ansatz)
    return ansatz

In [ ]:
from pytket.utils.operators import QubitPauliOperator
from pytket.utils import get_operator_expectation_value

hamiltonian_op = QubitPauliOperator.from_OpenFermion(hamiltonian)

# Simplified objective function using utilities
def objective(params):
    circ = ucc(params)
    return get_operator_expectation_value(circ, hamiltonian_op, backend, n_shots=4000) + nuclear_repulsion_energy

In [ ]:
arg_values = [-3.79002933e-05, 2.42964799e-05, 4.63447157e-01]

energy = objective(arg_values)
print(energy)

-1.13977910049376


# Measuring efficiency in terms of gate reductions

In [ ]:
from pytket import OpType
from pytket.passes import FullPeepholeOptimise
test_circuit = ucc(arg_values)

print("CX count before", test_circuit.n_gates_of_type(OpType.CX))
print("CX depth before", test_circuit.depth_by_type(OpType.CX))

FullPeepholeOptimise().apply(test_circuit)

print("CX count after FPO", test_circuit.n_gates_of_type(OpType.CX))
print("CX depth after FPO", test_circuit.depth_by_type(OpType.CX))

CX count before 56
CX depth before 52
CX count after FPO 30
CX depth after FPO 28


In [ ]:
from pytket.passes import PauliSimp
test_circuit = ucc(arg_values)

print("CX count before", test_circuit.n_gates_of_type(OpType.CX))
print("CX depth before", test_circuit.depth_by_type(OpType.CX))

PauliSimp().apply(test_circuit)

print("CX count after PS", test_circuit.n_gates_of_type(OpType.CX))
print("CX depth after PS", test_circuit.depth_by_type(OpType.CX))

FullPeepholeOptimise().apply(test_circuit)

print("CX count after PS+FPO", test_circuit.n_gates_of_type(OpType.CX))
print("CX depth after PS+FPO", test_circuit.depth_by_type(OpType.CX))

CX count before 56
CX depth before 52
CX count after PS 30
CX depth after PS 25
CX count after PS+FPO 21
CX depth after PS+FPO 19


In [ ]:
# Including the process of circuit simplification and optimisation into our objective function
def objective(params):
    circ = ucc(params)
    PauliSimp().apply(circ)
    FullPeepholeOptimise().apply(circ)
    return get_operator_expectation_value(circ, hamiltonian_op, backend, n_shots=4000) + nuclear_repulsion_energy

# Including further tket optimizations to reduce the amount of computation

In [ ]:
from pytket.passes import GuidedPauliSimp
from pytket.utils import gen_term_sequence_circuit

def ucc(params):
    singles_params = {qps : params[0] * coeff for qps, coeff in singles.items()}
    doubles_params = {qps : params[1] * coeff for qps, coeff in doubles.items()}
    excitation_op = QubitPauliOperator({**singles_params, **doubles_params})
    reference_circ = Circuit(4).X(1).X(3)
    ansatz = gen_term_sequence_circuit(excitation_op, reference_circ)
    GuidedPauliSimp().apply(ansatz)
    FullPeepholeOptimise().apply(ansatz)
    return ansatz

In [ ]:
from sympy import symbols

# Symbolic UCC ansatz generation
syms = symbols("p0 p1 p2")
singles_a_syms = {qps : syms[0] * coeff for qps, coeff in singles_a.items()}
singles_b_syms = {qps : syms[1] * coeff for qps, coeff in singles_b.items()}
doubles_syms = {qps : syms[2] * coeff for qps, coeff in doubles.items()}
excitation_op = QubitPauliOperator({**singles_a_syms, **singles_b_syms, **doubles_syms})
ucc_ref = Circuit(4).X(1).X(3)
ucc = gen_term_sequence_circuit(excitation_op, ucc_ref)
GuidedPauliSimp().apply(ucc)
FullPeepholeOptimise().apply(ucc)

# Objective function using the symbolic ansatz
def objective(params):
    circ = ucc.copy()
    sym_map = dict(zip(syms, params))
    circ.symbol_substitution(sym_map)
    return get_operator_expectation_value(circ, hamiltonian_op, backend, n_shots=4000) + nuclear_repulsion_energy


# defining the most optimized objective function

In [ ]:
from pytket.partition import PauliPartitionStrat

# Objective function using measurement reduction
def objective(params):
    circ = ucc.copy()
    sym_map = dict(zip(syms, params))
    circ.symbol_substitution(sym_map)
    return get_operator_expectation_value(
        circ, operator, backend,
        n_shots=4000,
        partition_strat=PauliPartitionStrat.CommutingSets,
    ) + nuclear_repulsion_energy

# Final tket optimized VQE running on IBM Aer simulator

In [ ]:
from openfermion import QubitOperator
from scipy.optimize import minimize
from sympy import symbols

from pytket.backends.ibm import AerBackend
from pytket.circuit import Circuit, Qubit
from pytket.partition import PauliPartitionStrat
from pytket.passes import GuidedPauliSimp, FullPeepholeOptimise
from pytket.pauli import Pauli, QubitPauliString
from pytket.utils import get_operator_expectation_value, gen_term_sequence_circuit
from pytket.utils.operators import QubitPauliOperator

# Obtain electronic Hamiltonian
hamiltonian = (-0.8153001706270075 * QubitOperator('') +
    0.16988452027940318 * QubitOperator('Z0') +
    -0.21886306781219608 * QubitOperator('Z1') +
    0.16988452027940323 * QubitOperator('Z2') +
    -0.2188630678121961 * QubitOperator('Z3') +
    0.12005143072546047 * QubitOperator('Z0 Z1') +
    0.16821198673715723 * QubitOperator('Z0 Z2') +
    0.16549431486978672 * QubitOperator('Z0 Z3') +
    0.16549431486978672 * QubitOperator('Z1 Z2') +
    0.1739537877649417 * QubitOperator('Z1 Z3') +
    0.12005143072546047 * QubitOperator('Z2 Z3') +
    0.04544288414432624 * QubitOperator('X0 X1 X2 X3') +
    0.04544288414432624 * QubitOperator('X0 X1 Y2 Y3') +
    0.04544288414432624 * QubitOperator('Y0 Y1 X2 X3') +
    0.04544288414432624 * QubitOperator('Y0 Y1 Y2 Y3'))
nuclear_repulsion_energy = 0.70556961456

hamiltonian_op = QubitPauliOperator.from_OpenFermion(hamiltonian)

# Obtain terms for single and double excitations
q = [Qubit(i) for i in range(4)]
xyii = QubitPauliString([q[0], q[1]], [Pauli.X, Pauli.Y])
yxii = QubitPauliString([q[0], q[1]], [Pauli.Y, Pauli.X])
iixy = QubitPauliString([q[2], q[3]], [Pauli.X, Pauli.Y])
iiyx = QubitPauliString([q[2], q[3]], [Pauli.Y, Pauli.X])
xxxy = QubitPauliString(q, [Pauli.X, Pauli.X, Pauli.X, Pauli.Y])
xxyx = QubitPauliString(q, [Pauli.X, Pauli.X, Pauli.Y, Pauli.X])
xyxx = QubitPauliString(q, [Pauli.X, Pauli.Y, Pauli.X, Pauli.X])
yxxx = QubitPauliString(q, [Pauli.Y, Pauli.X, Pauli.X, Pauli.X])
yyyx = QubitPauliString(q, [Pauli.Y, Pauli.Y, Pauli.Y, Pauli.X])
yyxy = QubitPauliString(q, [Pauli.Y, Pauli.Y, Pauli.X, Pauli.Y])
yxyy = QubitPauliString(q, [Pauli.Y, Pauli.X, Pauli.Y, Pauli.Y])
xyyy = QubitPauliString(q, [Pauli.X, Pauli.Y, Pauli.Y, Pauli.Y])

# Symbolic UCC ansatz generation
syms = symbols("p0 p1 p2")
singles_syms = {xyii : syms[0], yxii : -syms[0], iixy : syms[1], iiyx : -syms[1]}
doubles_syms = {
    xxxy : 0.25*syms[2],
    xxyx : -0.25*syms[2],
    xyxx : 0.25*syms[2],
    yxxx : -0.25*syms[2],
    yyyx : -0.25*syms[2],
    yyxy : 0.25*syms[2],
    yxyy : -0.25*syms[2],
    xyyy : 0.25*syms[2],
}
excitation_op = QubitPauliOperator({**singles_syms, **doubles_syms})
ucc_ref = Circuit(4).X(0).X(2)
ucc = gen_term_sequence_circuit(excitation_op, ucc_ref)

# Circuit simplification
GuidedPauliSimp().apply(ucc)
FullPeepholeOptimise().apply(ucc)

# Connect to a simulator/device
backend = AerBackend()

# Objective function
def objective(params):
    circ = ucc.copy()
    sym_map = dict(zip(syms, params))
    circ.symbol_substitution(sym_map)
    return get_operator_expectation_value(
        circ, hamiltonian_op, backend,
        n_shots=4000,
        partition_strat=PauliPartitionStrat.CommutingSets,
    ) + nuclear_repulsion_energy

# Optimise against the objective function
initial_params = [1e-4, 1e-4, 4e-1]
result = minimize(objective, initial_params, method="Nelder-Mead")
print("Final parameter values", result.x)
print("Final energy value", result.fun)

Final parameter values [ 0.00013154  0.00015386 -0.03824673]
Final energy value -1.1489176373381045


# Final tket optimized VQE running on Qulacs NISQ simulator

In [ ]:
from openfermion import QubitOperator
from scipy.optimize import minimize
from sympy import symbols
from pytket.backends.qulacs import QulacsBackend

from pytket.backends.ibm import AerBackend
from pytket.circuit import Circuit, Qubit
from pytket.partition import PauliPartitionStrat
from pytket.passes import GuidedPauliSimp, FullPeepholeOptimise
from pytket.pauli import Pauli, QubitPauliString
from pytket.utils import get_operator_expectation_value, gen_term_sequence_circuit
from pytket.utils.operators import QubitPauliOperator

# Obtain electronic Hamiltonian
hamiltonian = (-0.8153001706270075 * QubitOperator('') +
    0.16988452027940318 * QubitOperator('Z0') +
    -0.21886306781219608 * QubitOperator('Z1') +
    0.16988452027940323 * QubitOperator('Z2') +
    -0.2188630678121961 * QubitOperator('Z3') +
    0.12005143072546047 * QubitOperator('Z0 Z1') +
    0.16821198673715723 * QubitOperator('Z0 Z2') +
    0.16549431486978672 * QubitOperator('Z0 Z3') +
    0.16549431486978672 * QubitOperator('Z1 Z2') +
    0.1739537877649417 * QubitOperator('Z1 Z3') +
    0.12005143072546047 * QubitOperator('Z2 Z3') +
    0.04544288414432624 * QubitOperator('X0 X1 X2 X3') +
    0.04544288414432624 * QubitOperator('X0 X1 Y2 Y3') +
    0.04544288414432624 * QubitOperator('Y0 Y1 X2 X3') +
    0.04544288414432624 * QubitOperator('Y0 Y1 Y2 Y3'))
nuclear_repulsion_energy = 0.70556961456

hamiltonian_op = QubitPauliOperator.from_OpenFermion(hamiltonian)

# Obtain terms for single and double excitations
q = [Qubit(i) for i in range(4)]
xyii = QubitPauliString([q[0], q[1]], [Pauli.X, Pauli.Y])
yxii = QubitPauliString([q[0], q[1]], [Pauli.Y, Pauli.X])
iixy = QubitPauliString([q[2], q[3]], [Pauli.X, Pauli.Y])
iiyx = QubitPauliString([q[2], q[3]], [Pauli.Y, Pauli.X])
xxxy = QubitPauliString(q, [Pauli.X, Pauli.X, Pauli.X, Pauli.Y])
xxyx = QubitPauliString(q, [Pauli.X, Pauli.X, Pauli.Y, Pauli.X])
xyxx = QubitPauliString(q, [Pauli.X, Pauli.Y, Pauli.X, Pauli.X])
yxxx = QubitPauliString(q, [Pauli.Y, Pauli.X, Pauli.X, Pauli.X])
yyyx = QubitPauliString(q, [Pauli.Y, Pauli.Y, Pauli.Y, Pauli.X])
yyxy = QubitPauliString(q, [Pauli.Y, Pauli.Y, Pauli.X, Pauli.Y])
yxyy = QubitPauliString(q, [Pauli.Y, Pauli.X, Pauli.Y, Pauli.Y])
xyyy = QubitPauliString(q, [Pauli.X, Pauli.Y, Pauli.Y, Pauli.Y])

# Symbolic UCC ansatz generation
syms = symbols("p0 p1 p2")
singles_syms = {xyii : syms[0], yxii : -syms[0], iixy : syms[1], iiyx : -syms[1]}
doubles_syms = {
    xxxy : 0.25*syms[2],
    xxyx : -0.25*syms[2],
    xyxx : 0.25*syms[2],
    yxxx : -0.25*syms[2],
    yyyx : -0.25*syms[2],
    yyxy : 0.25*syms[2],
    yxyy : -0.25*syms[2],
    xyyy : 0.25*syms[2],
}
excitation_op = QubitPauliOperator({**singles_syms, **doubles_syms})
ucc_ref = Circuit(4).X(0).X(2)
ucc = gen_term_sequence_circuit(excitation_op, ucc_ref)

# Circuit simplification
GuidedPauliSimp().apply(ucc)
FullPeepholeOptimise().apply(ucc)

# Connect to a simulator/device
backend = QulacsBackend()

# Objective function
def objective(params):
    circ = ucc.copy()
    sym_map = dict(zip(syms, params))
    circ.symbol_substitution(sym_map)
    return get_operator_expectation_value(
        circ, hamiltonian_op, backend,
        n_shots=4000,
        partition_strat=PauliPartitionStrat.CommutingSets,
    ) + nuclear_repulsion_energy

# Optimise against the objective function
initial_params = [1e-4, 1e-4, 4e-1]
result = minimize(objective, initial_params, method="Nelder-Mead")
print("Final parameter values", result.x)
print("Final energy value", result.fun)

Final parameter values [ 0.00015757  0.00013549 -0.045553  ]
Final energy value -1.1525988847400837
